In [1]:
from __future__ import print_function


import os
import subprocess

import pandas as pd
import numpy as np
import math
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import svm
from sklearn.cluster import KMeans
from sklearn.cross_validation import train_test_split
import threading
import itertools
import time
from sklearn.cross_validation import StratifiedShuffleSplit
import matplotlib.pyplot as plt


#from unbalanced_dataset import NearMiss, CondensedNearestNeighbour, OneSidedSelection,\
#NeighbourhoodCleaningRule, TomekLinks, ClusterCentroids, OverSampler, SMOTE,\
#SMOTETomek, SMOTEENN, EasyEnsemble, BalanceCascade

#UnderSampler, 







In [2]:
def findRenewalProb(value, leafid):    
    summation = value[leafid][0][0] + value[leafid][0][1]
    probability = (value[leafid][0][0] / summation) * 100
    return probability
	
def recurse(left, right, threshold, features, node_number, value, tree, f):
    global stack
    if (threshold[node_number] != -2):
        stack.append("{0} <= {1}".format(features[node_number], threshold[node_number]))
        recurse(left, right, threshold, features, left[node_number], value, tree, f)
        stack.pop()
        stack.append("{0} > {1}".format(features[node_number], threshold[node_number]))
        recurse(left, right, threshold, features, right[node_number], value, tree, f)
        stack.pop()
    else:   
        f.write(str(node_number) + "," + str(value[node_number][0][0]) + "," + str(value[node_number][0][1]))
        prob = findRenewalProb(value, node_number)
        f.write(",{0},".format(value[node_number][0][0] + value[node_number][0][1]) + "{0: .4f}%,".format(prob))
        for item in stack:
            f.write("," + item )
        f.write("\n")
		
#Traversing the tree and giving Dynamic Insights

def traverse(tree, feature_names):
    
    left = tree.tree_.children_left
    right = tree.tree_.children_right
    threshold = tree.tree_.threshold
    features  = [feature_names[i] for i in tree.tree_.feature]
    value = tree.tree_.value
    f = open('leaf_of_DT_for_analysis.csv', 'wb')
    f.write("LeafID,RenewedCount,ChurnedCount,TotalCount,RenewalProbability,Items\n")
    recurse(left, right, threshold, features, 0, value, tree, f)
    f.close()





def getListFirstElement(lst):
	return lst[0]

def visualize_tree(tree, feature_names,id):
	"""Create tree png using graphviz.

	Args
	----
	tree -- scikit-learn DecsisionTree.
	feature_names -- list of feature names.
	"""
	print("debug")
	# dot_file = str(id)+".dot"
	# png_file = str(id)+".png"
    # with open(dot_file, 'w') as f:
		# export_graphviz(tree, out_file=f,
						# feature_names=features,
						# #class_names=Targets,  
						# filled=True, rounded=True,  
						# #special_characters=True,
						# leaves_parallel = True,
						# label = 'all')
						# #class_names = ["Churned","Renewed"])
						
	# #command = ["dot", "-Tpng", "dtWithCorrectlabels_allfeatures05-02-2016.dot", "-o", "dtWithCorrectlabels_allfeatures05-02-2016.png"]
	# command = ["dot", "-Tpng", dot_file, "-o", png_file]
	# try:
		# subprocess.check_call(command)
	# except:
		# exit("Could not run dot, ie graphviz, to "
			 # "produce visualization")


			 
# Reading data from file.
#


In [3]:
import datetime
from datetime import datetime


df_main = pd.read_csv("C:/Users/rishab/Desktop/FinalDataSet.csv", encoding='ISO-8859-1')

df_main.fillna(inplace = True, value = 0)

# Features Dictionary


AgeGroup = [str(i) for i in np.sort(df_main.AgeGroup.unique())]
channelName = [str(i) for i in np.sort(df_main.channelName.unique())]
#platform = [str(i) for i in np.sort(df_main.platform.unique()) ]
gender = [str(i) for i in np.sort(df_main.gender.unique())]
callIntro = [str(i) for i in np.sort(df_main.callIntro.unique())]
#userWeightTarget = [str(i) for i in np.sort(df_main.userWeightTarget.unique())]
channelType = [str(i) for i in np.sort(df_main.channelType.unique())]
#city = [str(i) for i in np.sort(df_main.city.unique())]
IsProfilePicSet = [str(i) for i in np.sort(df_main.IsProfilePicSet.unique())]
engagement = [str(i) for i in np.sort(df_main.engagement.unique())]

list_of_featuresUniqueValueLists = [AgeGroup,channelName,gender,IsProfilePicSet,callIntro,engagement,channelType]
feature_names = ['AgeGroup','channelName','gender','IsProfilePicSet','callIntro','engagement','channelType']


for i in range(len(list_of_featuresUniqueValueLists)):
	map_to_int                =     {name:list_of_featuresUniqueValueLists[i].index(name) for name in list_of_featuresUniqueValueLists[i]}
	featureName               =     feature_names[i]
	featureNumberString       =     featureName + "Number"
	#print (featureNumberString)
	df_main[featureNumberString]  =  df_main[featureName].replace(map_to_int)
	#df_future[featureNumberString]  =  df_future[featureName].replace(map_to_int)


df_sample = df_main[df_main["Status"] != "SubscriptionEndsAfterToday" ]
df_future = df_main[df_main["Status"] == "SubscriptionEndsAfterToday"]

df_sample.reset_index(inplace = True, drop = True)
df_future.reset_index(inplace = True, drop = True)

df_future["last_use_date"] = pd.to_datetime(df_future["last_use_date"], format = "%Y-%m-%d")

df_future = df_future[df_future["last_use_date"] < datetime.strptime('01072016', "%d%m%Y").date()]
df_future.reset_index(inplace = True , drop = True)

targets = [str(i) for i in np.sort(df_sample.Status.unique())]
# print ("debug")
# print (targets)
	
#Target
# map_to_int = {name: n for n, name in enumerate(targets)}
map_to_int = {name:targets.index(name) for name in targets}

df_sample["Target"] = df_sample['Status'].replace(map_to_int)

				

final_set_columns = df_future.columns

df_set = pd.DataFrame(columns = final_set_columns)

df_set_1stLowest = pd.DataFrame(columns = final_set_columns)
df_set_2ndLowest = pd.DataFrame(columns = final_set_columns)




all_features = ["calls_firstMonthValue",           
              "score_firstMonthValue","coach_avg_sentence_length",
              "coach_links_per_message","coach_uniq_words_per_message",
              "player_avg_sentence_length","turn_count",
              "EP_firstMonthValue", "EP_secondMonthValue", "EP_secondLastMonthValue",              
              "foodLog_firstMonthValue","foodLog_secondMonthValue",
              "foodLog_secondLastMonthValue","stepsScore_firstMonthValue",          
              "stepsScore_secondMonthValue","stepsScore_secondLastMonthValue",         
              "totalDeviceSleep_firstMonthValue","totalDeviceSleep_secondMonthValue","totalDeviceSleep_secondLastMonthValue",
              "waterScore_firstMonthValue","waterScore_secondMonthValue",        
              "waterScore_secondLastMonthValue",
              "coach_emoji_score","emoji_score_normalized","emoji_score_per_message","player_emoji_score"]

feature_combination_set_big = [all_features]


features = feature_combination_set_big[0]

feature_future = [
# last month featuresturn_count','canned_ratio','coach_char_count','player_total_messages','waterScore_lastMonthValue','stepsScore_lastMonthValue',
   'waterScore_lastMonthValue','stepsScore_lastMonthValue',
    'foodLog_lastMonthValue','EP_lastMonthValue','score_lastMonthValue','systemActive_lastMonthValue','conversationActive_lastMonthValue',
'totalDeviceSleep_lastMonthValue', 'userId'

]


score_cutoffs = 0.2


for feature_combination_set in feature_combination_set_big:
	#print ("GOne into not monthly")
	features_for_model = feature_combination_set
	# Seperating the training and testing set using Scikit Learn.
	# And iterating for many combinations.
	
	# Number of clusters for the risk bucketing
	
	# df_training = pd.concat(training_dfs)
	# df_testing = pd.concat(testing_dfs)
	
	df_sample_renewed = df_sample[df_sample["Status"] == "Renewed"]
	df_sample_churned = df_sample[df_sample["Status"] == "churned"]
	
	num_of_trees = (df_sample_churned.shape[0] * 1.0 ) / df_sample_renewed.shape[0]
	
	if (num_of_trees - int(num_of_trees) > 0.5):
		num_of_trees = int(num_of_trees) + 1
	else:
		num_of_trees = int(num_of_trees)
	
	df_sample_sets =  np.array_split(df_sample_churned, num_of_trees)
	
	for i in range(0,len(df_sample_sets)):
		df_sample_sets[i] = pd.concat([df_sample_sets[i], df_sample_renewed] , ignore_index = True)
    
	
	X= df_sample[features_for_model]
	y= df_sample['Target']
	
	dt_min_split = 30

	classifiers = []
	for i in range(num_of_trees):
		classifiers.append(DecisionTreeClassifier(min_samples_split=dt_min_split))
	#dt = DecisionTreeClassifier(min_samples_split=dt_min_split , random_state=99)

	for i in range(len(classifiers)):
		X= df_sample_sets[i][features_for_model]
		Y= df_sample_sets[i]['Target']
		classifiers[i].fit(X,Y)
        print(classifiers[i].tree_.value)
		# dt.fit(X, y)
	
	# print (df_training[features_for_model[len(features_for_model)-1]].unique())
	map_userId_to_score = {}
	
	print("Step2 done")
	
	# getting the renewal score from the DecisionTree.
	for id in df_sample.userId.tolist():
		features_for_id = np.array(df_sample.ix[df_sample.userId == id,features_for_model].iloc[0].tolist())
		# print (features_for_id)
		
		predict_array = []
		for i in range(len(classifiers)):
			if ( classifiers[i].predict(features_for_id.reshape(1,-1)) == targets.index("churned") ):
				predict_array.append(1)
			else:
				predict_array.append(0)
		
		renewal_score = np.mean(predict_array)
		
		map_userId_to_score.__setitem__(id,renewal_score)
		
		# X.reshape(1, -1)

	df_sample['renewal_score'] = df_sample['userId'].replace(map_userId_to_score)

	print("Step3 done")

	'''
	# Predicting for future.
	'''

	map_userId_to_score = {}
    #diction={}

	# getting the renewal score from the DecisionTree.
	
        #diction.__setitem__(id,predict_array)
		
		# X.reshape(1, -1)

	
##here samyak and rishab







C:\Users\rishab\Anaconda2\lib\site-packages\ipykernel\__main__.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\rishab\Anaconda2\lib\site-packages\ipykernel\__main__.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


[[[   0.    0.]]

 [[   0.    0.]]

 [[   0.    0.]]

 [[   0.    0.]]

 [[   0.    0.]]

 [[   0.    0.]]

 [[   2.    0.]]

 [[   0.    0.]]

 [[   0.    0.]]

 [[   1.    0.]]

 [[   0.    0.]]

 [[   0.    0.]]

 [[   0.    0.]]

 [[   1.    1.]]

 [[   0.    0.]]

 [[   0.    0.]]

 [[   0.    0.]]

 [[   0.    0.]]

 [[   0.    0.]]

 [[   0.    0.]]

 [[   0.    0.]]

 [[   0.   38.]]

 [[   5.   11.]]

 [[   0.   58.]]

 [[   0.    0.]]

 [[   0.    0.]]

 [[   1.   17.]]

 [[   0.   32.]]

 [[   0.  182.]]

 [[   2.   12.]]

 [[   1.    2.]]

 [[   1.    2.]]

 [[   0.    0.]]

 [[   0.    0.]]

 [[   3.   21.]]

 [[   4.    2.]]

 [[   0.   33.]]

 [[   0.    0.]]

 [[   0.    0.]]

 [[   7.   17.]]

 [[   0.    0.]]

 [[   1.    2.]]

 [[   0.    0.]]

 [[   0.   33.]]

 [[   1.    6.]]

 [[   2.    0.]]

 [[   1.    0.]]

 [[   1.    0.]]

 [[   0.    0.]]

 [[   0.    0.]]

 [[   1.    0.]]

 [[   0.    0.]]

 [[   0.    0.]]

 [[   1.    0.]]

 [[   0.    0.]]

 [[   0.  

C:\Users\rishab\Anaconda2\lib\site-packages\ipykernel\__main__.py:166: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [4]:
from collections import defaultdict
diction={}

In [5]:

for id in df_future.userId.tolist():
        
	features_for_id = np.array(df_future.ix[df_future.userId == id,features_for_model].iloc[0].tolist())
		# print (features_for_id)
		###print (features_for_id)
		
	predict_array = []
	for i in range(len(classifiers)):
		if ( classifiers[i].predict(features_for_id.reshape(1,-1)) == targets.index("churned") ):
			predict_array.append(1)
		else:
			predict_array.append(0)

	renewal_score = np.mean(predict_array)

	map_userId_to_score.__setitem__(id,renewal_score)
        diction.__setitem__(id,predict_array)

In [6]:
df_future['renewal_score'] = df_future['userId'].replace(map_userId_to_score)

print("Step4 done")

	
	# Mapping clusters on the testing data.
map_userId_to_propensityBucket = {}

for id in df_future.userId.tolist():
	
	renewal_score = df_future.ix[df_future.userId == id,'renewal_score'].iloc[0]
		
	if (renewal_score > score_cutoffs):
		renewal_propensity_bucket = 0
	else:
		renewal_propensity_bucket = 1
		
	map_userId_to_propensityBucket.__setitem__(id,renewal_propensity_bucket)

df_future['Renewal Propensity Bucket'] = df_future['userId'].replace(map_userId_to_propensityBucket)
	
print("Step5 done")
	
print("Step6 done")
	
	
df_set_1stLowest = pd.concat( [ df_set_1stLowest , df_future[df_future['Renewal Propensity Bucket']  == max(df_future['Renewal Propensity Bucket']) ][final_set_columns] ] )
		
	

df_set_1stLowest.drop_duplicates(inplace = True)


Step4 done
Step5 done
Step6 done


In [15]:
print(map_userId_to_propensityBucket)

{106497: 0, 106498: 0, 106499: 0, 106503: 0, 106509: 0, 106510: 0, 106514: 0, 106515: 0, 106516: 0, 106519: 0, 106520: 0, 106525: 1, 106526: 0, 106527: 0, 114720: 0, 106531: 0, 106532: 1, 106533: 0, 106534: 0, 106536: 0, 106537: 0, 106539: 0, 106544: 0, 106546: 0, 104750: 0, 106549: 0, 106550: 0, 106551: 0, 106554: 1, 106555: 0, 106558: 0, 106561: 0, 106566: 0, 87393: 0, 106568: 1, 106569: 0, 87394: 1, 106574: 0, 106576: 1, 106577: 0, 106578: 0, 106579: 0, 106580: 0, 82007: 0, 90201: 0, 106586: 1, 106588: 1, 90205: 0, 106591: 0, 106596: 0, 106597: 0, 106598: 0, 106605: 0, 106606: 0, 106607: 1, 106608: 0, 106610: 0, 106616: 0, 113343: 0, 106620: 0, 106621: 0, 106623: 0, 106624: 0, 106625: 0, 106629: 0, 106632: 1, 106634: 0, 106636: 0, 106638: 0, 106641: 0, 106642: 0, 106645: 0, 106646: 1, 110617: 0, 106649: 0, 106650: 0, 106651: 0, 106653: 0, 106654: 0, 106656: 0, 106657: 0, 106658: 0, 106659: 1, 106661: 0, 106664: 0, 106668: 0, 90285: 0, 106674: 0, 180: 0, 106677: 0, 106678: 1, 106679:

In [7]:

dictlist=[]
def traversetrain(tree, feature_names, target_names, feature_values,userid):#function that appends all renewed userids to the leaf


    left = tree.tree_.children_left
    right = tree.tree_.children_right
    threshold = tree.tree_.threshold
    featuresid = tree.tree_.feature
    features  = [feature_names[i] for i in tree.tree_.feature]
    value = tree.tree_.value
    leafid_original = tree.tree_.apply(np.array(feature_values,dtype=('float32')).reshape(1,-1))[0]
    userids[leafid_original].append(userid)

In [20]:
for i in range(0,len(df_sample_sets)):
    userids=defaultdict(list)
    for j in range(0,len(df_sample_renewed)):
        sample = df_sample_renewed[j:j+1]
        userid = sample['userId'].values[0]
        vals = sample[all_features].values
        traversetrain(classifiers[i], all_features, targets, vals[0],userid)
    dictlist.append(userids)
#print(vals[0])
#print(vals)

In [22]:


from __future__ import division 
def perc(userid):
    pervalues=defaultdict(list)
    
    userg=pd.read_csv("C:/Users/rishab/Desktop/insightGoqii3/USER"+str(userid)+".csv")
    usergmeans=pd.read_csv("C:/Users/rishab/Desktop/insightGoqii3/USER"+str(userid)+"means"+".csv")
    



    
    
    



    for feat in feature_future:
        if(feat=='userId'):
            break
        usergsort=userg.sort(columns=feat)
        k=usergmeans[feat+"currentuservalue"][0]
        count=0
        for index,row in usergsort.iterrows():
            if(row[feat]<k):
                count=count+1
        
        
        
            else:
                
                pervalues[feat+'percentilescore'].append((((count/(len(usergsort)))*100)))
                
                break
    percalc=pd.DataFrame(pervalues)
    percalc.reset_index(inplace = True, drop = True)
    usergmeans.reset_index(inplace = True, drop = True)
    final=pd.concat([usergmeans,percalc], axis=1)
    final.to_csv("C:/Users/rishab/Desktop/insightGoqii3/USER"+str(userid)+"means"+".csv") 
#print(count)

In [26]:

from compiler.ast import flatten
mon=[]
def traversetest( feature_names, target_names, feature_values,userid,dect,y_newfeature):#function that finds the leaf where the currentuser lands and returns the average of all renewed people in that leaf   
    sum=0
    df1=dect[y_newfeature]
    #df1.to_csv("C:/Users/rishab/Desktop/insightGoqii/USER"+str(userid)+".csv")
    leafid_original=[]
    n=len(y_newfeature)-1
    means=[]
    meanscurrent=[]
    listofallid=[]
    setlist=[]
    tuplelist=[]
    #print
    prcount=0
   
    if(map_userId_to_propensityBucket[userid]==0):    
      
        for i in range(0,num_of_trees):
            if(diction[userid][i]==1):
                prcount=prcount+1
        
        
        
                mea=[]
                meacurrent=[]
                leafidoriginal=classifiers[i].tree_.apply(np.array(feature_values,dtype=('float32')).reshape(1,-1))[0]
        
        
                listofallid.append(dictlist[i][leafidoriginal])
    
                k1=pd.DataFrame()
                currentk1=pd.DataFrame()
                frames=[]
                emptyarray=[]
   
                currentframes=[]
                n=len(y_newfeature)-1
        

                if(dictlist[i].has_key(leafidoriginal)):

                    for i in dictlist[i][leafidoriginal]:
                        emptyarray.append(i)

                        frames.append(df1[df1['userId']==i])
        
                    currentframes.append(df1[df1['userId']==userid])
                    currentk1=pd.concat(currentframes)
            
                
        

                    if not emptyarray:
           
                        for j in range(0,n):
                             mea.append(0)
                        means.append(mea)
                
                
            
           
                    else:
                        k1=pd.concat(frames)
                
                
            
                
           
                
                        for j in range(0,n):
                    
                            mea.append(k1[y_newfeature[j]].mean())
                        means.append(mea)
                
                else:
        
                    print("no matching data found")
    else:
        for i in range(0,num_of_trees):
            if(diction[userid][i]==0):
                prcount=prcount+1
        
        
        
                mea=[]
                meacurrent=[]
                leafidoriginal=classifiers[i].tree_.apply(np.array(feature_values,dtype=('float32')).reshape(1,-1))[0]
        
        
                listofallid.append(dictlist[i][leafidoriginal])
        
                k1=pd.DataFrame()
                currentk1=pd.DataFrame()
                frames=[]
                emptyarray=[]
   
                currentframes=[]
                n=len(y_newfeature)-1
        

                if(dictlist[i].has_key(leafidoriginal)):

                    for i in dictlist[i][leafidoriginal]:
                        emptyarray.append(i)

                        frames.append(df1[df1['userId']==i])
        
                    currentframes.append(df1[df1['userId']==userid])
                    currentk1=pd.concat(currentframes)
            
                
        

                    if not emptyarray:
           
                        for j in range(0,n):
                             mea.append(0)
                        means.append(mea)
                
                
            
           
                    else:
                        k1=pd.concat(frames)
                
                
            
                
           
                
                        for j in range(0,n):
                    
                            mea.append(k1[y_newfeature[j]].mean())
                        means.append(mea)
                
                else:
        
                    print("no matching data found")
        
    
    consolidatedmean=[]
    conslist=[]
    
    
    
    conslist=set(list(flatten(listofallid)))
    dafr=pd.DataFrame()
    framesnew=[]
    for i in conslist:
        framesnew.append(df1[df1['userId']==i])
    if(len(framesnew)==0):
        return 0
    
    
    dafr=pd.concat(framesnew)
    
    dafrmean=pd.DataFrame()
    
            
        
    
    dafr.to_csv("C:/Users/rishab/Desktop/insightGoqii3/USER"+str(userid)+".csv")
    print(len(set(list(flatten(listofallid)))))
   

    

    
    
        


    f = open("C:/Users/rishab/Desktop/insightGoqii3/USER"+str(userid)+"means"+".csv", "wb")
    f.write("userid"+',')
        
    
    
    
    for i in range(0,len(feature_future)-1):
        f.write(str(feature_future[i])+',')
    
    for i in range(0,len(feature_future)-1):
        f.write(str(feature_future[i])+"currentuservalue"',')
    f.write("\n")
    f.write(str(userid)+',')
    
    for j in range(0,n):
        sum=0
        
        for i in range(0,(prcount)):
            
            sum=sum+means[i][j]
            meant=sum/num_of_trees
        
    
    
        f.write(str(meant) + ",")
   
    
    
    for j in range(0,n):
        f.write(str(currentk1[y_newfeature[j]].mean())+",")
    f.close()
        
    perc(userid)
   
        

        
    
    


In [27]:

for i in range(0,len(df_set_1stLowest)):
    sample = df_set_1stLowest[i:i+1]
    userid = sample['userId'].values[0]
    vals = sample[features].values
    
    traversetest( features, targets, vals[0],userid,df_main,feature_future)
    
    
                
        

        



  






443
604
293
605
524
638
639
63
606
666
305
295
637
555
585
804
581
449
220
518
635
678
505
620
575
109
412
300
573
435
195
290
765
610
577
671
562
459
770
103
770
669
667
246
767
540
609
688
769
696
459
660
619
475
271
692
727
725
333
631
560
810
647
769
784
723
728
279
818
728
759
751
285
701
745
788
778
764
748
582
687
733
781
578
617
727
658
726
613
713
726
760
621
371
710
570
657
611
509
435
555
705
594
723
303
651
561
385
306
552
517
356
361
476
246
553
782
749
174
495
708
396
226
730
544
620
692
432
724
319
599
787
773
755
708
307
264
665
618
614
793
635
595
640
243
251
619
568
643
650
785
433
732
674
136
366
621
588
731
505
550
712
772
775
527
456
654
255
794
769
585
233
265
763
797
728
658
546
573
601
678
663
738
767
607
764
587
589
583
412
675
545
734
590
730
595
713
660
394
666
426
554
645
793
721
713
666
706
768
738
667
525
588
496
789
632
697
750
378
755
558
586
586
47
794
764
736
729
734
717
610
645
716
698
533
643
735
767
130
727
627
363
508
597
504
259
560
568
585
288
31

C:\Users\rishab\Anaconda2\lib\site-packages\ipykernel\__main__.py:22: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [37]:
print(mon)

[2037.8700717816703, 1138.86567598603, 1.8181818181818181, 2276.924535578016, 906.7461225311176, 2250.9565569569527, 119.30695620169303, 508.1695258247343]


In [23]:
a=[1,4,3,6,7]

In [24]:
b=set(a)

In [25]:
b

{1, 3, 4, 6, 7}

In [72]:
df_main.groupby('Status').size()

Status
Renewed                        1106
SubscriptionEndsAfterToday     8823
churned                       14848
dtype: int64

In [1]:
dictlist = []

In [2]:
dictlist.append({'key':'rishab'})

In [3]:
dictlist.append({'key':'chocolate'})

In [4]:
dictlist.append({'key':'bus'})

In [6]:
dictlist

[{'key': 'rishab'}, {'key': 'chocolate'}, {'key': 'bus'}]

In [7]:
dictlist[0]

{'key': 'rishab'}

In [8]:
dictlist[1]

{'key': 'chocolate'}

In [25]:
dictlist[0]

defaultdict(list,
            {7: [83754, 116547, 116604],
             10: [95499],
             15: [84820, 92488, 93315, 108634, 111604],
             19: [80327, 103542],
             26: [115164],
             28: [82896, 100022],
             29: [80600,
              81391,
              83400,
              83747,
              88094,
              88303,
              90695,
              91265,
              92040,
              94024,
              98386,
              100739,
              101666,
              102748,
              103982,
              111637,
              111680,
              111700,
              111702,
              111877,
              111980,
              113517],
             32: [98743],
             33: [81409,
              83671,
              83929,
              88913,
              88957,
              90047,
              90153,
              90346,
              90594,
              93215,
              98855,
              100397,
   

In [26]:
for k in dictlist[0][7]:
    print(k)

83754
116547
116604


In [16]:

from collections import defaultdict


a=[2,3,4,5,6,7,8,8,9,9,10]


In [17]:
for i in a :
    user['5'].append(i)

In [18]:
user

defaultdict(list, {'5': [2, 3, 4, 5, 6, 7, 8, 8, 9, 9, 10]})

In [26]:
maps={}
maps.__setitem__('5',a)


In [27]:
maps

{'5': [2, 3, 4, 5, 6, 7, 8, 8, 9, 9, 10]}

In [87]:
import matplotlib as mpl
%matplotlib inline 
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf
from matplotlib.backends.backend_pdf import PdfPages


# create an empty pdf file


# last month featuresturn_count','canned_ratio','coach_char_count','player_total_messages','waterScore_lastMonthValue','stepsScore_lastMonthValue',
   
feature_future = [
# last month featuresturn_count','canned_ratio','coach_char_count','player_total_messages','waterScore_lastMonthValue','stepsScore_lastMonthValue',
 'waterScore_lastMonthValue','stepsScore_lastMonthValue',
    'foodLog_lastMonthValue','EP_lastMonthValue','score_lastMonthValue','systemActive_lastMonthValue','conversationActive_lastMonthValue',
'totalDeviceSleep_lastMonthValue'

]


def sorted(userid):

    
    with PdfPages("C:/Users/rishab/Desktop/insightimages3/insight"+str(userid)+".pdf") as pdf:
        for feat in feature_future:
            gg=pd.read_csv("C:/Users/rishab/Desktop/insightGoqii3/USER"+str(userid)+".csv")
            gg1=['userId']
            gg1.append(feat)
            g=pd.read_csv("C:/Users/rishab/Desktop/insightGoqii3/USER"+str(userid)+"means"+".csv")
            gg2=['userid']
            gg2.append(feat)
            gg3=gg.sort(columns=feat)
            
            if(len(gg3)>50):
                gg4=pd.DataFrame(columns=[feat])
                
                             
                feat2=feat+"currentuservalue"
                gg3=gg3[gg3[feat]!=0]
                gg3.sort(columns=feat).plot(y=feat,kind='bar')
                x=plt.axhline(y=g[feat2][0],color='red' ).get_figure()
                pdf.savefig(x)
                plt.close()
            else:    
                
            

                gg11=gg[gg1]
                gg22=g[gg2]
    #gg5=gg.sort(columns=feat)
                feat2=feat+"currentuservalue"
                gg.sort(columns=feat).plot(y=feat,kind='bar')
    
#k=g['turn_count']
                x=plt.axhline(y=g[feat2][0],color='red' ).get_figure()
                pdf.savefig(x)
                plt.close()
            
            
    #plt.axhline(y=g[feat][0],color='red')
    #plt.savefig("C:/Users/rishab/Desktop/insightGoqii/USER456"+".png")   # save the figure to file
    #plt.close(fig)    # close the figure
    
#a=gg[gg1].plot(y='turn_count')
#g[gg1].plot(axis=a)
#g['turn_count'].plot(x='userid',y='turn_count',kind='line')
#plt.show()
#g['turn_count'][0]

In [88]:
userss=[103724,
        117449,
        116469,
        116374,
        116323,
        116155,
        116108,
        115993,
        115802,
        113005,
        111707,
        108963,
        106842,
        106664,
        106586,
        106444,
        106363,
        106345,
        106315,
        106265,
        106193,
        106175,
        106147,
        106097,
        106077,
        106067,
        106047,
        105918,
        105739,
        105658,
        105648,
        105500,
        105487,
        105479,
        105451,
        105258,
        105038,
        104841,
        104834,
        104734,
        104719,
        104703,
        
        104663,
        104633,
        104558,
        104548,
        104501,
        104498,
        104431,
        104117,
        104094]


In [90]:
kdf=df_set_1stLowest[df_set_1stLowest['userId'].isin(userss)]
kdf.reset_index(inplace=True,drop=True)
for i in range(21,25):
    sample = kdf[i:i+1]
    userid = sample['userId'].values[0]
    sorted(userid)
    print(userid)


106077.0
106097.0
106147.0
106193.0


C:\Users\rishab\Anaconda2\lib\site-packages\ipykernel\__main__.py:33: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
C:\Users\rishab\Anaconda2\lib\site-packages\ipykernel\__main__.py:41: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
